In [1]:
import socket
import threading
# import tcp_server
from datetime import datetime

In [2]:
class TCPServer:
    ''' A simple TCP Server for handling a single client '''

    def __init__(self, host, port):
        self.host = host            # Host address
        self.port = port            # Host port
        self.sock = None            # Connection socket

    def printwt(self, msg):
        ''' Print message with current date and time '''

        current_date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        print(f'[{current_date_time}] {msg}')

    def configure_server(self):
        ''' Configure the server '''

        # create TCP socket with IPv4 addressing
        self.printwt('Creating socket...')
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.printwt('Socket created')

        # bind server to the address
        self.printwt(f'Binding server to {self.host}:{self.port}...')
        self.sock.bind((self.host, self.port))
        self.printwt(f'Server binded to {self.host}:{self.port}')

    def wait_for_client(self):
        ''' Wait for a client to connect '''

        # start listening for incoming connections
        self.printwt('Listening for incoming connection...')
        self.sock.listen(1)

        # accept a connection
        client_sock, client_address = self.sock.accept()
        self.printwt(f'Accepted connection from {client_address}')
        self.handle_client(client_sock, client_address)

    def get_phone_no(self, name):
        ''' Get phone no for a given name '''

        phonebook = {'Alex': '1234567890', 'Bob': '1234512345'}

        if name in phonebook.keys():
            return f"{name}'s phone number is {phonebook[name]}"
        else:
            return f"No records found for {name}"

    def handle_client(self, client_sock, client_address):
        """ Handle the accepted client's requests """

        try:
            data_enc = client_sock.recv(1024)
            while data_enc:
                # client's request
                name = data_enc.decode()
                resp = self.get_phone_no(name)
                self.printwt(f'[ REQUEST from {client_address} ]')
                print('\n', name, '\n')

                # send response
                self.printwt(f'[ RESPONSE to {client_address} ]')
                client_sock.sendall(resp.encode('utf-8'))
                print('\n', resp, '\n')

                # get more data and check if client closed the connection
                data_enc = client_sock.recv(1024)
            self.printwt(f'Connection closed by {client_address}')

        except OSError as err:
            self.printwt(err)

        finally:
            self.printwt(f'Closing client socket for {client_address}...')
            client_sock.close()
            self.printwt(f'Client socket closed for {client_address}')

    def shutdown_server(self):
        ''' Shutdown the server '''

        self.printwt('Shutting down server...')
        self.sock.close()

In [ ]:

class TCPServerMultiClient(TCPServer):
    ''' A simple TCP Server for handling multiple clients '''

    def __init__(self, host, port):
        super().__init__(host, port)
        self.client_list=[]

    def wait_for_client(self):
        ''' Wait for clients to connect '''

        try:
            self.printwt('Listening for incoming connection')
            self.sock.listen(10) # 10 clients before server refuses connections
            #blocking or non-blocking questions doesn't arise because each socket is handled in different thread
            #hence, if recv is blocked, then it'll be in that thread and doesn't affect others
            #internally threads are kind of polled

            while True:

                client_sock, client_address = self.sock.accept()
                self.client_list.append(client_sock)
                self.printwt(f'Accepted connection from {client_address}')
                c_thread = threading.Thread(target = self.handle_client,
                                        args = (client_sock, client_address))
                c_thread.start()

        except KeyboardInterrupt:
            self.shutdown_server()
            
    def handle_client(self, client_sock, client_address):
        try:
            data_received = client_sock.recv(100)
            while data_received:
                self.printwt("msg from Client{} is {}".format(self.client_list.index(client_sock),data_received.decode('utf-8')))
                reply = "Hello Client, your data is received"
                client_sock.send(reply.encode('utf-8'))
                data_received = client_sock.recv(100)
               
                
                
                
        except OSError as err:
            self.printwt(err)

        finally:
#             self.printwt(f'Closing client socket for {client_address}...')
            client_sock.close()
            self.printwt(f'Client socket closed for {client_address}')
        

def main():
    ''' Create a TCP Server and handle multiple clients simultaneously '''

    tcp_server_multi_client = TCPServerMultiClient('127.0.0.1', 4444)
    tcp_server_multi_client.configure_server()
    #wait_for_client has infinite while loop
    tcp_server_multi_client.wait_for_client()

if __name__ == '__main__':
    main()

[2021-05-09 11:40:41.566] Creating socket...
[2021-05-09 11:40:41.566] Socket created
[2021-05-09 11:40:41.567] Binding server to 127.0.0.1:4444...
[2021-05-09 11:40:41.567] Server binded to 127.0.0.1:4444
[2021-05-09 11:40:41.567] Listening for incoming connection
[2021-05-09 11:40:44.991] Accepted connection from ('127.0.0.1', 51078)
[2021-05-09 11:40:50.982] msg from Client0 is sending2
[2021-05-09 11:40:50.986] msg from Client0 is sending2
[2021-05-09 11:40:50.987] msg from Client0 is sending2
[2021-05-09 11:40:50.988] msg from Client0 is sending2
[2021-05-09 11:40:50.988] msg from Client0 is sending2
[2021-05-09 11:40:50.989] msg from Client0 is sending2
[2021-05-09 11:40:50.990] msg from Client0 is sending2
[2021-05-09 11:40:50.990] msg from Client0 is sending2
[2021-05-09 11:40:50.991] msg from Client0 is sending2
[2021-05-09 11:40:50.992] msg from Client0 is sending2
[2021-05-09 11:40:59.981] msg from Client0 is sending2
[2021-05-09 11:40:59.982] msg from Client0 is sending2
[2

[2021-05-09 11:43:15.628] msg from Client0 is sending2
[2021-05-09 11:43:15.628] msg from Client0 is sending2
[2021-05-09 11:43:15.628] msg from Client0 is sending2
[2021-05-09 11:43:15.635] msg from Client0 is sending2
[2021-05-09 11:43:24.847] msg from Client0 is sending2
[2021-05-09 11:43:29.577] msg from Client0 is sending2
[2021-05-09 11:43:29.577] msg from Client0 is sending2
[2021-05-09 11:43:29.582] msg from Client0 is sending2
[2021-05-09 11:43:29.582] msg from Client0 is sending2
[2021-05-09 11:43:29.585] msg from Client0 is sending2
[2021-05-09 11:43:29.586] msg from Client0 is sending2
[2021-05-09 11:43:29.586] msg from Client0 is sending2
[2021-05-09 11:43:29.587] msg from Client0 is sending2
[2021-05-09 11:43:29.587] msg from Client0 is sending2
[2021-05-09 11:43:29.587] msg from Client0 is sending2
[2021-05-09 11:43:29.588] msg from Client0 is sending2
[2021-05-09 11:43:29.588] msg from Client0 is sending2
[2021-05-09 11:43:29.589] msg from Client0 is sending2
[2021-05-0

[2021-05-09 11:47:06.076] Client socket closed for ('127.0.0.1', 51078)
[2021-05-09 11:47:10.997] Accepted connection from ('127.0.0.1', 51108)
[2021-05-09 11:48:14.601] msg from Client1 is sending2
[2021-05-09 11:48:14.606] msg from Client1 is sending2
[2021-05-09 11:48:14.607] msg from Client1 is sending2
[2021-05-09 11:48:14.609] msg from Client1 is sending2
[2021-05-09 11:48:14.610] msg from Client1 is sending2
[2021-05-09 11:48:14.611] msg from Client1 is sending2
[2021-05-09 11:48:14.612] msg from Client1 is sending2
[2021-05-09 11:48:14.613] msg from Client1 is sending2
[2021-05-09 11:48:14.614] msg from Client1 is sending2
[2021-05-09 11:48:14.614] msg from Client1 is sending2
[2021-05-09 11:48:14.617] msg from Client1 is sending2
[2021-05-09 11:48:14.619] msg from Client1 is sending2
[2021-05-09 11:48:14.619] msg from Client1 is sending2
[2021-05-09 11:48:14.620] msg from Client1 is sending2
[2021-05-09 11:48:14.620] msg from Client1 is sending2
[2021-05-09 11:48:14.620] msg f

[2021-05-09 11:48:31.167] msg from Client1 is sending2
[2021-05-09 11:48:31.167] msg from Client1 is sending2
[2021-05-09 11:48:31.167] msg from Client1 is sending2
[2021-05-09 11:48:31.174] msg from Client1 is sending2
[2021-05-09 11:48:31.177] msg from Client1 is sending2
[2021-05-09 11:48:31.179] msg from Client1 is sending2
[2021-05-09 11:48:31.180] msg from Client1 is sending2
[2021-05-09 11:48:31.180] msg from Client1 is sending2
[2021-05-09 11:48:31.182] msg from Client1 is sending2
[2021-05-09 11:48:31.182] msg from Client1 is sending2
[2021-05-09 11:48:31.184] msg from Client1 is sending2
[2021-05-09 11:48:31.184] msg from Client1 is sending2
[2021-05-09 11:48:31.185] msg from Client1 is sending2
[2021-05-09 11:48:31.186] msg from Client1 is sending2
[2021-05-09 11:48:31.186] msg from Client1 is sending2
[2021-05-09 11:48:31.190] msg from Client1 is sending2
[2021-05-09 11:48:31.192] msg from Client1 is sending2
[2021-05-09 11:48:31.193] msg from Client1 is sending2
[2021-05-0

[2021-05-09 11:48:39.065] msg from Client1 is sending2
[2021-05-09 11:48:39.070] msg from Client1 is sending2
[2021-05-09 11:48:39.072] msg from Client1 is sending2
[2021-05-09 11:48:39.074] msg from Client1 is sending2
[2021-05-09 11:48:39.077] msg from Client1 is sending2
[2021-05-09 11:48:39.078] msg from Client1 is sending2
[2021-05-09 11:48:39.081] msg from Client1 is sending2
[2021-05-09 11:48:39.083] msg from Client1 is sending2
[2021-05-09 11:48:39.085] msg from Client1 is sending2
[2021-05-09 11:48:39.089] msg from Client1 is sending2
[2021-05-09 11:48:39.090] msg from Client1 is sending2
[2021-05-09 11:48:39.091] msg from Client1 is sending2
[2021-05-09 11:48:39.093] msg from Client1 is sending2
[2021-05-09 11:48:39.095] msg from Client1 is sending2
[2021-05-09 11:48:39.096] msg from Client1 is sending2
[2021-05-09 11:48:39.096] msg from Client1 is sending2
[2021-05-09 11:48:39.096] msg from Client1 is sending2
[2021-05-09 11:48:39.096] msg from Client1 is sending2
[2021-05-0

[2021-05-09 14:41:48.867] Client socket closed for ('127.0.0.1', 51108)
[2021-05-09 14:44:22.914] Accepted connection from ('127.0.0.1', 51604)
[2021-05-09 14:44:32.067] msg from Client2 is sending2
[2021-05-09 14:44:32.071] msg from Client2 is sending2
[2021-05-09 14:44:32.073] msg from Client2 is sending2
[2021-05-09 14:44:32.075] msg from Client2 is sending2
[2021-05-09 14:44:32.077] msg from Client2 is sending2
[2021-05-09 14:44:32.077] msg from Client2 is sending2
[2021-05-09 14:44:32.078] msg from Client2 is sending2
[2021-05-09 14:44:32.080] msg from Client2 is sending2
[2021-05-09 14:44:32.081] msg from Client2 is sending2
[2021-05-09 14:44:32.082] msg from Client2 is sending2
[2021-05-09 14:44:32.083] msg from Client2 is sending2
[2021-05-09 14:44:32.086] msg from Client2 is sending2
[2021-05-09 14:44:32.086] msg from Client2 is sending2
[2021-05-09 14:44:32.087] msg from Client2 is sending2
[2021-05-09 14:44:32.088] msg from Client2 is sending2
[2021-05-09 14:44:32.088] msg f

[2021-05-09 15:51:43.580] msg from Client2 is sending2
[2021-05-09 15:51:43.580] msg from Client2 is sending2
[2021-05-09 15:51:43.584] msg from Client2 is sending2
[2021-05-09 15:51:43.584] msg from Client2 is sending2
[2021-05-09 15:51:43.586] msg from Client2 is sending2
[2021-05-09 15:51:43.588] msg from Client2 is sending2
[2021-05-09 15:51:43.589] msg from Client2 is sending2
[2021-05-09 15:51:43.590] msg from Client2 is sending2
[2021-05-09 15:51:43.591] msg from Client2 is sending2
[2021-05-09 15:51:43.593] msg from Client2 is sending2
[2021-05-09 15:51:43.594] msg from Client2 is sending2
[2021-05-09 15:51:43.595] msg from Client2 is sending2
[2021-05-09 15:51:43.598] msg from Client2 is sending2
[2021-05-09 15:51:43.598] msg from Client2 is sending2
[2021-05-09 15:51:43.599] msg from Client2 is sending2
[2021-05-09 15:51:43.603] msg from Client2 is sending2
[2021-05-09 15:51:43.605] msg from Client2 is sending2
[2021-05-09 15:51:43.605] msg from Client2 is sending2
[2021-05-0

[2021-05-09 15:51:51.731] Client socket closed for ('127.0.0.1', 51604)
